# TF Transformer standalone
Example with TF Transformer in standalone mode using apache beam.

### Notes
To download the dataset follow the [link](https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers?hl=it)

In [1]:
import apache_beam as beam

import logging
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
import typing
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.ml.transforms.base import MLTransform
from apache_beam.ml.transforms.tft import ScaleToZScore, ComputeAndApplyVocabulary
import pathlib
import pprint
import tempfile
import csv

import tensorflow as tf
import tensorflow_transform as tft

import tensorflow_transform.beam as tft_beam
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils
from tensorflow_transform.keras_lib import tf_keras

logger = logging.getLogger()
logger.setLevel(logging.INFO)

csv_path = Path("data/petfinder") / "petfinder-mini.csv"

df = pd.read_csv(csv_path)
df.info()

2024-05-17 15:35:41.311327: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           11537 non-null  object
 1   Age            11537 non-null  int64 
 2   Breed1         11537 non-null  object
 3   Gender         11537 non-null  object
 4   Color1         11537 non-null  object
 5   Color2         11537 non-null  object
 6   MaturitySize   11537 non-null  object
 7   FurLength      11537 non-null  object
 8   Vaccinated     11537 non-null  object
 9   Sterilized     11537 non-null  object
 10  Health         11537 non-null  object
 11  Fee            11537 non-null  int64 
 12  Description    11528 non-null  object
 13  PhotoAmt       11537 non-null  int64 
 14  AdoptionSpeed  11537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB


In [2]:
schema = list(df.columns)
def _set_types(d):
    """ Set feature types """
    d['Age'] = int(d['Age'])
    d['Fee'] = int(d['Fee'])
    d['PhotoAmt'] = int(d['PhotoAmt'])
    d['AdoptionSpeed'] = int(d['AdoptionSpeed'])
    return d

counter = 0
def filter_first_n_rows(row, n: int =5) -> bool:
    """ Extract the first n rows."""
    global counter
    if counter < n:
        counter += 1
        return True
    else:
        return False

def parse_csv(line) -> dict:
    """ Convert line of the csv row to dict """
    return dict(zip(schema, next(csv.reader([line]))))

with (beam.Pipeline() as pipeline):
    lines = (
            pipeline
            | beam.io.textio.ReadFromText(str(csv_path), skip_header_lines=1) # .with_input_types(PetFinderSchema)
            | beam.Map(parse_csv)
            | beam.Map(_set_types)
            # | MLTransform(write_artifact_location=artifact_location).with_transform(scale_age).with_transform(tf_idf_descr)
            | beam.Filter(lambda x: filter_first_n_rows(x, n=3))
            | beam.Map(print)
    )

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.


INFO:apache_beam.runners.worker.statecache:Creating state cache with size 104857600


{'Type': 'Cat', 'Age': 3, 'Breed1': 'Tabby', 'Gender': 'Male', 'Color1': 'Black', 'Color2': 'White', 'MaturitySize': 'Small', 'FurLength': 'Short', 'Vaccinated': 'No', 'Sterilized': 'No', 'Health': 'Healthy', 'Fee': 100, 'Description': "Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of cats a few months ago but could not get them neutered in time as the clinic was fully scheduled. The result was this little kitty. I do not have enough space and funds to care for more cats in my household. Looking for responsible people to take over Nibble's care.", 'PhotoAmt': 1, 'AdoptionSpeed': 2}
{'Type': 'Cat', 'Age': 1, 'Breed1': 'Domestic Medium Hair', 'Gender': 'Male', 'Color1': 'Black', 'Color2': 'Brown', 'MaturitySize': 'Medium', 'FurLength': 'Medium', 'Vaccinated': 'Not Sure', 'Sterilized': 'Not Sure', 'Health': 'Healthy', 'Fee': 0, 'Description': 'I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to provide

In [3]:
df.select_dtypes(exclude='number').nunique()

Type                2
Breed1            166
Gender              2
Color1              7
Color2              7
MaturitySize        3
FurLength           3
Vaccinated          3
Sterilized          3
Health              3
Description     10691
dtype: int64

In [4]:
schema = list(df.columns)
def _set_types(d):
    """ Set feature types """
    d['Age'] = int(d['Age'])
    d['Fee'] = int(d['Fee'])
    d['PhotoAmt'] = int(d['PhotoAmt'])
    d['AdoptionSpeed'] = int(d['AdoptionSpeed'])
    return d

counter = 0
def filter_first_n_rows(row, n: int =5) -> bool:
    """ Extract the first n rows."""
    global counter
    if counter < n:
        counter += 1
        return True
    else:
        return False

def parse_csv(line) -> dict:
    """ Convert line of the csv row to dict """
    return dict(zip(schema, next(csv.reader([line]))))

raw_data_metadata = dataset_metadata.DatasetMetadata(
    schema_utils.schema_from_feature_spec({
        'Type': tf.io.FixedLenFeature([], tf.string),
        'Age': tf.io.FixedLenFeature([], tf.float32),
        'Breed1': tf.io.FixedLenFeature([], tf.string),
        'Gender': tf.io.FixedLenFeature([], tf.string),
        'Color1': tf.io.FixedLenFeature([], tf.string),
        'Color2': tf.io.FixedLenFeature([], tf.string),
        'MaturitySize': tf.io.FixedLenFeature([], tf.string),
        'FurLength': tf.io.FixedLenFeature([], tf.string),
        'Vaccinated': tf.io.FixedLenFeature([], tf.string),
        'Sterilized': tf.io.FixedLenFeature([], tf.string),
        'Health': tf.io.FixedLenFeature([], tf.string),
        'Fee': tf.io.FixedLenFeature([], tf.float32),
        'Description': tf.io.VarLenFeature(tf.string),
        'PhotoAmt': tf.io.FixedLenFeature([], tf.float32),
        'AdoptionSpeed': tf.io.FixedLenFeature([], tf.float32),
    }))


def binarize_adoption_speed_tfop(input_value):
  """
  Binarize the adoption speed feature
  """
  return tf.reshape(tf.where(input_value==4, 1.0, 0.0), [-1])

def cat_to_one_hot_tfop(input_value, vocab_filename='x_vocab'):
    """
    Convert categorical feature to one-hot feature
    """
    integerized = tft.compute_and_apply_vocabulary(
      input_value,
      num_oov_buckets=1,
      vocab_filename=vocab_filename)
    one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(tft.experimental.get_vocabulary_size_by_name(vocab_filename) + 1,
                    tf.int32),
      on_value=1.0,
      off_value=0.0)
    return one_hot_encoded

def text_to_tfidf_tfops(input_value):
  integerized = tft.compute_and_apply_vocabulary(
      input_value)
  vocab_size = tft.get_num_buckets_for_transformed_feature(integerized)
  vocab_index, tfidf_weight = tft.tfidf(integerized, vocab_size)


def preprocessing_fn(inputs):
  return {
      'type': cat_to_one_hot_tfop(inputs["Type"], vocab_filename='petfinder_type'),
      'age': tft.scale_to_z_score(inputs['Age']),
      #'breed': inputs["Breed1"],
      'gender': cat_to_one_hot_tfop(inputs["Gender"], vocab_filename='petfinder_gender'),
      'color1': cat_to_one_hot_tfop(inputs["Color1"], vocab_filename='petfinder_color1'),
      'color2': cat_to_one_hot_tfop(inputs["Color2"], vocab_filename='petfinder_color2'),
      'maturity_size': cat_to_one_hot_tfop(inputs["MaturitySize"], vocab_filename='petfinder_maturitysize'),
      'fur_length': cat_to_one_hot_tfop(inputs["FurLength"], vocab_filename='petfinder_furlegth'),
      'vaccinated': cat_to_one_hot_tfop(inputs["Vaccinated"], vocab_filename='petfinder_vaccinated'),
      'sterilized': cat_to_one_hot_tfop(inputs["Sterilized"], vocab_filename='petfinder_sterilized'),
      'health': cat_to_one_hot_tfop(inputs["Health"], vocab_filename='petfinder_health'),
      'fee': tft.scale_to_z_score(inputs["Fee"]),
      # 'description': inputs["Description"],
      'photo_amt': tft.scale_to_z_score(inputs["PhotoAmt"]),
      'adoption_speed': binarize_adoption_speed_tfop(inputs["AdoptionSpeed"])
  }

# raw_data = [
#     {'Type': 'Cat', 'Age': 3, 'Breed1': 'Tabby', 'Gender': 'Male', 'Color1': 'Black', 'Color2': 'White', 'MaturitySize': 'Small', 'FurLength': 'Short', 'Vaccinated': 'No', 'Sterilized': 'No', 'Health': 'Healthy', 'Fee': 100, 'Description': "Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of cats a few months ago but could not get them neutered in time as the clinic was fully scheduled. The result was this little kitty. I do not have enough space and funds to care for more cats in my household. Looking for responsible people to take over Nibble's care.", 'PhotoAmt': 1, 'AdoptionSpeed': 2},
#     {'Type': 'Cat', 'Age': 1, 'Breed1': 'Domestic Medium Hair', 'Gender': 'Male', 'Color1': 'Black', 'Color2': 'Brown', 'MaturitySize': 'Medium', 'FurLength': 'Medium', 'Vaccinated': 'Not Sure', 'Sterilized': 'Not Sure', 'Health': 'Healthy', 'Fee': 0, 'Description': 'I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to provide temporary care.', 'PhotoAmt': 2, 'AdoptionSpeed': 0},
#     {'Type': 'Dog', 'Age': 1, 'Breed1': 'Mixed Breed', 'Gender': 'Male', 'Color1': 'Brown', 'Color2': 'White', 'MaturitySize': 'Medium', 'FurLength': 'Medium', 'Vaccinated': 'Yes', 'Sterilized': 'No', 'Health': 'Healthy', 'Fee': 0, 'Description': 'Their pregnant mother was dumped by her irresponsible owner at the roadside near some shops in Subang Jaya. Gave birth to them at the roadside. They are all healthy and adorable puppies. Already dewormed, vaccinated and ready to go to a home. No tying or caging for long hours as guard dogs. However, it is acceptable to cage or tie for precautionary purposes. Interested to adopt pls call me.', 'PhotoAmt': 7, 'AdoptionSpeed': 3},
#     {'Type': 'Dog', 'Age': 4, 'Breed1': 'Mixed Breed', 'Gender': 'Female', 'Color1': 'Black', 'Color2': 'Brown', 'MaturitySize': 'Medium', 'FurLength': 'Short', 'Vaccinated': 'Yes', 'Sterilized': 'No', 'Health': 'Healthy', 'Fee': 150, 'Description': 'Good guard dog, very alert, active, obedience waiting for her good master, plz call or sms for more details if you really get interested, thanks!!', 'PhotoAmt': 8, 'AdoptionSpeed': 2},
#     {'Type': 'Dog', 'Age': 1, 'Breed1': 'Mixed Breed', 'Gender': 'Male', 'Color1': 'Black', 'Color2': 'No Color', 'MaturitySize': 'Medium', 'FurLength': 'Short', 'Vaccinated': 'No', 'Sterilized': 'No', 'Health': 'Healthy', 'Fee': 0, 'Description': "This handsome yet cute boy is up for adoption. He is the most playful pal we've seen in our puppies. He loves to nibble on shoelaces , Chase you at such a young age. Imagine what a cute brat he will be when he grows. We are looking for a loving home for Hunter , one that will take care of him and give him the love that he needs. Please call urgently if you would like to adopt this cutie.", 'PhotoAmt': 3, 'AdoptionSpeed': 2}
# ]

pipeline_options = beam.pipeline.PipelineOptions(runner='Direct')

with beam.Pipeline(options=pipeline_options) as pipeline:
    raw_data = (
            pipeline
            | beam.io.textio.ReadFromText(str(csv_path), skip_header_lines=1) # .with_input_types(PetFinderSchema)
            | beam.Map(parse_csv)
            | beam.Map(_set_types)
            # | MLTransform(write_artifact_location=artifact_location).with_transform(scale_age).with_transform(tf_idf_descr)
            | beam.Filter(lambda x: filter_first_n_rows(x, n=3))
    )

    with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
        transformed_dataset, transform_fn = (  # pylint: disable=unused-variable
            (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(
                preprocessing_fn))

        transformed_data, transformed_metadata = transformed_dataset
        _ = transformed_data | beam.Map(print)


INFO:absl:Feature AdoptionSpeed has a shape . Setting to DenseTensor.
INFO:absl:Feature Age has a shape . Setting to DenseTensor.
INFO:absl:Feature Breed1 has a shape . Setting to DenseTensor.
INFO:absl:Feature Color1 has a shape . Setting to DenseTensor.
INFO:absl:Feature Color2 has a shape . Setting to DenseTensor.
INFO:absl:Feature Description has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Fee has a shape . Setting to DenseTensor.
INFO:absl:Feature FurLength has a shape . Setting to DenseTensor.
INFO:absl:Feature Gender has a shape . Setting to DenseTensor.
INFO:absl:Feature Health has a shape . Setting to DenseTensor.
INFO:absl:Feature MaturitySize has a shape . Setting to DenseTensor.
INFO:absl:Feature PhotoAmt has a shape . Setting to DenseTensor.
INFO:absl:Feature Sterilized has a shape . Setting to DenseTensor.
INFO:absl:Feature Type has a shape . Setting to DenseTensor.
INFO:absl:Feature Vaccinated has a shape . Setting to DenseTensor.
INFO:absl:Feature Adopt

INFO:tensorflow:Assets written to: /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp7gop5xms/tftransform_tmp/4506437cc44a4a5395b8bd647613eb74/assets


INFO:tensorflow:Assets written to: /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp7gop5xms/tftransform_tmp/4506437cc44a4a5395b8bd647613eb74/assets


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:root:BatchElements statistics: element_count=3 batch_count=3 next_batch_size=1 timings=[(1, 0.0071620941162109375), (1, 0.008028745651245117)]
INFO:root:BatchElements statistics: element_count=3 batch_count=3 next_batch_size=1 timings=[(1, 0.001627206802368164), (1, 0.0018041133880615234)]
INFO:root:BatchElements statistics: element_count=2 batch_count=1 next_batch_size=150000 timings=[]
INFO:root:BatchElements statistics: element_count=1 batch_count=1 next_batch_size=150000 timings=[]
INFO:root:BatchElements statistics: element_count=2 batch_count=1 next_batch_size=150000 timings=[]
INFO:root:BatchElements statistics: element_count=2 batch_count=1 next_batch_size=150000 timings=[]
INFO:root:BatchElements statistics: element_count=2 batch_count=1 next_batch_size=150000 timings=[]
INFO:root:BatchElements statistics: element_count=2 batch_count=1 next_batch_size=150000 timings=[]
INFO:root:BatchElements statistics: element_count=3 ba

INFO:tensorflow:Assets written to: /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp7gop5xms/tftransform_tmp/1c1549a098de4311961a8c9c2d64f745/assets


INFO:tensorflow:Assets written to: /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp7gop5xms/tftransform_tmp/1c1549a098de4311961a8c9c2d64f745/assets


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


{'adoption_speed': 0.0, 'age': 1.4142136573791504, 'color1': array([1., 0., 0.], dtype=float32), 'color2': array([1., 0., 0.], dtype=float32), 'fee': 1.4142136573791504, 'fur_length': array([0., 1., 0.], dtype=float32), 'gender': array([1., 0.], dtype=float32), 'health': array([1., 0.], dtype=float32), 'maturity_size': array([0., 1., 0.], dtype=float32), 'photo_amt': -0.8890008330345154, 'sterilized': array([1., 0., 0.], dtype=float32), 'type': array([1., 0., 0.], dtype=float32), 'vaccinated': array([0., 0., 1., 0.], dtype=float32)}
{'adoption_speed': 0.0, 'age': -0.7071067094802856, 'color1': array([1., 0., 0.], dtype=float32), 'color2': array([0., 1., 0.], dtype=float32), 'fee': -0.7071067094802856, 'fur_length': array([1., 0., 0.], dtype=float32), 'gender': array([1., 0.], dtype=float32), 'health': array([1., 0.], dtype=float32), 'maturity_size': array([1., 0., 0.], dtype=float32), 'photo_amt': -0.5080004930496216, 'sterilized': array([0., 1., 0.], dtype=float32), 'type': array([1.,